In [ ]:
!git clone https://github.com/sipeed/Maix_Toolbox
!mkdir Maix_Toolbox/workspace
!mkdir Maix_Toolbox/ncc
%cd /content/Maix_Toolbox/ncc
!wget https://github.com/kendryte/nncase/releases/download/v0.1.0-rc5/ncc-linux-x86_64.tar.xz
!tar Jxf ncc-linux-x86_64.tar.xz
%cd /content/Maix_Toolbox/workspace

In [ ]:
%tensorflow_version 1.x

In [ ]:
from __future__ import print_function
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D
from tensorflow.keras import backend as K
import math
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import layers

np.random.seed(1337)
(x_values, y_values), (x_test, y_test) = mnist.load_data()
x_values = x_values / x_values.max()


In [ ]:
x_values = x_values.reshape(x_values.shape[0], 28, 28, 1)
x_values = tf.image.resize(x_values, [14,14])
sess=tf.Session()
with sess.as_default():
    x_values = x_values.eval()
x_values.max()

In [ ]:
TRAIN_SPLIT = int(0.6 * len(x_values))
TEST_SPLIT = int(0.2 * len(x_values) + TRAIN_SPLIT)
x_train, x_test, x_validate = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
y_train, y_test, y_validate = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Conv2D(4, (3, 3), activation='relu', padding='same', input_shape=(14, 14, 1)))
model.add(layers.Flatten())
model.add(layers.Dense(len(np.unique(y_train))))

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(x_train, y_train, epochs=12, batch_size=128, validation_data=(x_test, y_test))


In [ ]:
x_test = (x_test / x_test.max()).reshape((len(x_test), 14, 14, 1))
y_pred = model.predict(x_test).argmax(axis=1)

print('ACCURACY', (y_pred == y_test).sum() / len(y_test))

In [ ]:
%cd /content/Maix_Toolbox/
!mkdir images
import numpy as np
import cv2
batch_num=100
batch = x_train[0:batch_num]
imgs=batch.reshape((batch_num,14,14))*255
imgs=imgs.astype(np.uint8)
for i,img in enumerate(imgs):
  cv2.imwrite("images/%03d.jpg"%i, img)

jsontext="""
{
  "version": "0.1.0",
  "files": [
    {
      "address": 0x00300000,
      "bin": "mnist.kmodel",
      "sha256Prefix": false
    }
  ]
}
"""
with open("workspace/flash-list.json","w") as f:
  f.write(jsontext)

/content/Maix_Toolbox
mkdir: cannot create directory ‘images’: File exists


In [ ]:
%cd /content/Maix_Toolbox/workspace

In [ ]:
model.save("mnist.h5")

In [ ]:
%cd /content/Maix_Toolbox/
!tflite_convert  --output_file=/content/Maix_Toolbox/workspace/mnist.tflite --keras_model_file=/content/Maix_Toolbox/workspace/mnist.h5

In [ ]:
!./tflite2kmodel.sh workspace/mnist.tflite


In [ ]:
!./ncc/ncc -i tflite -o k210model --dataset /content/Maix_Toolbox/images /content/Maix_Toolbox/workspace/mnist.tflite /content/Maix_Toolbox/workspace/mnist.kmodel


In [ ]:
%cd /content/Maix_Toolbox/workspace


In [ ]:
!zip -r mnist_at_0x300000.kfpkg mnist.kmodel flash-list.json

In [ ]:
from google.colab import files
files.download('mnist_at_0x300000.kfpkg')

In [ ]:
#from google.colab import files
files.download('mnist.tflite')

In [ ]:
#from google.colab import files
files.download('mnist.h5')